In [ ]:
import os

import pandas as pd
from dotenv import load_dotenv

from sandbox.common.src.postgres_client import PostgresClient

load_dotenv()

In [ ]:
with PostgresClient(
    dbname=os.getenv("POSTGRES_DB", "postgres"),
    user=os.getenv("POSTGRES_USER", "admin"),
    password=os.getenv("POSTGRES_PASSWORD", "password"),
    host=os.getenv("POSTGRES_DB", "postgres"),
    port=5432,
) as db:
    db.execute("DROP TABLE IF EXISTS EMPLOYEES CASCADE")
    db.execute("DROP TABLE IF EXISTS ATTENDANCES CASCADE")

    db.execute(
        """
        CREATE TABLE IF NOT EXISTS EMPLOYEES (
            EMPLOYEE_ID SERIAL PRIMARY KEY,
            NAME TEXT NOT NULL
        );
        """
    )

    db.execute(
        """
        CREATE TABLE IF NOT EXISTS ATTENDANCES (
            ATTENDANCE_ID SERIAL PRIMARY KEY,
            EMPLOYEE_ID INTEGER NOT NULL REFERENCES EMPLOYEES (EMPLOYEE_ID),
            ATTENDANCE_DATE DATE NOT NULL
        );
        """
    )

    db.executemany(
        """
        INSERT INTO
            EMPLOYEES (NAME)
        VALUES
            (%s)
        """,
        [
            ("Alfa",),
            ("Bravo",),
            ("Charlie",),
        ],
    )

    db.executemany(
        """
        INSERT INTO
            ATTENDANCES (EMPLOYEE_ID, ATTENDANCE_DATE)
        VALUES
            (%s, CURRENT_DATE)
        """,
        [
            (1,),
            (2,),
        ],
    )

In [ ]:
with PostgresClient(
    dbname=os.getenv("POSTGRES_DB", "postgres"),
    user=os.getenv("POSTGRES_USER", "admin"),
    password=os.getenv("POSTGRES_PASSWORD", "password"),
    host=os.getenv("POSTGRES_DB", "postgres"),
    port=5432,
) as db:
    employees = db.fetchall("SELECT * FROM EMPLOYEES")
employees_df = pd.DataFrame(employees)
employees_df

In [ ]:
with PostgresClient(
    dbname=os.getenv("POSTGRES_DB", "postgres"),
    user=os.getenv("POSTGRES_USER", "admin"),
    password=os.getenv("POSTGRES_PASSWORD", "password"),
    host=os.getenv("POSTGRES_DB", "postgres"),
    port=5432,
) as db:
    attendances = db.fetchall("SELECT * FROM ATTENDANCES")
attendances_df = pd.DataFrame(attendances)
attendances_df

In [ ]:
with PostgresClient(
    dbname=os.getenv("POSTGRES_DB", "postgres"),
    user=os.getenv("POSTGRES_USER", "admin"),
    password=os.getenv("POSTGRES_PASSWORD", "password"),
    host=os.getenv("POSTGRES_DB", "postgres"),
    port=5432,
) as db:
    before_extract = db.fetchall(
        """
        SELECT
            CAST(EMPLOYEES.EMPLOYEE_ID AS INTEGER),
            EMPLOYEES.NAME,
            ATTENDANCES.ATTENDANCE_ID,
            ATTENDANCES.ATTENDANCE_DATE
        FROM
            EMPLOYEES
            LEFT JOIN ATTENDANCES ON EMPLOYEES.EMPLOYEE_ID = ATTENDANCES.EMPLOYEE_ID
        """
    )
before_extract_df = pd.DataFrame(before_extract)
before_extract_df

In [ ]:
with PostgresClient(
    dbname=os.getenv("POSTGRES_DB", "postgres"),
    user=os.getenv("POSTGRES_USER", "admin"),
    password=os.getenv("POSTGRES_PASSWORD", "password"),
    host=os.getenv("POSTGRES_DB", "postgres"),
    port=5432,
) as db:
    after_extract = db.fetchall(
        """
        SELECT
            CAST(EMPLOYEES.EMPLOYEE_ID AS INTEGER),
            EMPLOYEES.NAME,
            ATTENDANCES.ATTENDANCE_ID,
            ATTENDANCES.ATTENDANCE_DATE
        FROM
            EMPLOYEES
            LEFT JOIN ATTENDANCES ON EMPLOYEES.EMPLOYEE_ID = ATTENDANCES.EMPLOYEE_ID
        WHERE
            ATTENDANCES.EMPLOYEE_ID IS NULL
        """
    )
after_extract_df = pd.DataFrame(after_extract)
after_extract_df